## Dynamic Pricing - ANOVA Test - Sales Simulation


##### In this project I tried to apply ANOVA Test to find out optimum price of a product across different shops and simulate sales with different prices. Our goal is to ensure the maximum profit as predicted :)

##### The data we are going to use is consist of 2 columns which are Shop_id and Price.
##### The Shop_id represents 6 different shops and price is the amount that how much a customer wish to pay for a particular Item
##### Data set is consist of survey answers taken place different shops.
##### This project considers that survey participants answered the questions with rationality

### Imports

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import shapiro
import statsmodels.stats.api as sms

### Data Prepaeration

In [2]:
df= pd.read_csv("data.csv")

In [3]:
df.head()

,shop_id,price
0,shop_4,32.117753
1,shop_3,30.711370
2,shop_3,31.572607
3,shop_4,34.543840
4,shop_4,47.205824


In [4]:
df.shop_id.value_counts()

shop_4    1661
shop_6     733
shop_3     615
shop_2     144
shop_5     129
shop_1      97
Name: shop_id, dtype: int64

##### As we can see here the survey participants are not balanced between shops. But this is not a problem. ANOVA test is robust and can handle unequal number of observations problem.

##### Lets see what is the mean of price in different shops

In [5]:
df.groupby("shop_id").agg({"price": "mean"})

,price
shop_id,
shop_1,36.175498
shop_2,35.693170
shop_3,35.477261
shop_4,43.872913
shop_5,37.443592
shop_6,40.376575


##### Without the test shop_4 looks different than the other shops. But we will see 

In [6]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
price,3379.0,40.771387,19.730179,10.0,30.0,30.0,30.0,31.802522,34.744389,40.569827,56.921335,74.453248,135.848659,205.052944


##### When we look at the variance of price variable we can see that our assumption regarding answers of survey participants seems allright with a couple of exception.

##### Lets get rid of prices higher than 100

In [7]:
df=df[df.price <= 100]

In [8]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
price,3292.0,38.284972,11.657494,10.0,30.0,30.0,30.0,31.672294,34.678475,38.528301,53.188295,65.198083,88.622812,99.169972


##### Lets seperate shops for Testing

In [9]:
shop_1 = df[(df["shop_id"] == "shop_1")]["price"].values.tolist()
shop_2 = df[(df["shop_id"] == "shop_2")]["price"].values.tolist()
shop_3 = df[(df["shop_id"] == "shop_3")]["price"].values.tolist()
shop_4 = df[(df["shop_id"] == "shop_4")]["price"].values.tolist()
shop_5 = df[(df["shop_id"] == "shop_5")]["price"].values.tolist()
shop_6 = df[(df["shop_id"] == "shop_6")]["price"].values.tolist()

### One-Way ANOVA

##### Our hypothesises

##### H0 = Statisticly there is not a difference between prices accros shops
##### H1 = Statisticly there is a difference between prices accros shops

In [21]:
stats.f_oneway(shop_1, shop_2, shop_3, shop_4, shop_5, shop_6)

F_onewayResult(statistic=21.448116375929295, pvalue=3.4519617842416194e-21)

##### We rejected H0 hypothesis. Because, as you can see the p-value is an extremely small number

##### In short, Yes there is a statisticly difference between prices accros the shops but where this difference stem from. Which shop or shops are responsible from this difference. Lets find out

#### post-hoc test

##### Now lets check the prices in different shops wheter normally distributed or not. If it is normally distributed we will use independent t-test (parametric). If not we will use Mann–Whitney U test (non-parametric)


##### Our hypothesises
##### H0 = Price variable is normally distributed
##### H1 = Price variable is not normally distributed

In [10]:
shop_list = [shop_1,shop_2,shop_3,shop_4,shop_5,shop_6]
for i in shop_list:
    test_statistics, pvalue = shapiro(i)
    print('Test statistics = %.4f, p-value = %.4f' % (test_statistics, pvalue))

Test statistics = 0.6190, p-value = 0.0000
Test statistics = 0.5513, p-value = 0.0000
Test statistics = 0.4935, p-value = 0.0000
Test statistics = 0.7639, p-value = 0.0000
Test statistics = 0.6382, p-value = 0.0000
Test statistics = 0.6631, p-value = 0.0000



##### We rejected H0 hypothesis. Because, as you can see the p-values are smaller than "0,05"
##### In this case we need to use Mann–Whitney U test. If the data is not normally distributed we do not have to check the variance homogeneity

In [11]:
shop_names = ["shop_1","shop_2","shop_3","shop_4","shop_5","shop_6"] # shop names as string
shop_pairs = []

for first_shop in range(5):
    for second_shop  in range(first_shop+1,6):
        shop_pairs.append((shop_names[first_shop], shop_names[second_shop]))

##### As you can see here we created all possible combinations to perform a  Mann–Whitney U test

In [12]:
shop_pairs

[('shop_1', 'shop_2'),
 ('shop_1', 'shop_3'),
 ('shop_1', 'shop_4'),
 ('shop_1', 'shop_5'),
 ('shop_1', 'shop_6'),
 ('shop_2', 'shop_3'),
 ('shop_2', 'shop_4'),
 ('shop_2', 'shop_5'),
 ('shop_2', 'shop_6'),
 ('shop_3', 'shop_4'),
 ('shop_3', 'shop_5'),
 ('shop_3', 'shop_6'),
 ('shop_4', 'shop_5'),
 ('shop_4', 'shop_6'),
 ('shop_5', 'shop_6')]

In [13]:
test_stat = []
p_values = []
first_shop =[]
shop_names = ["shop_1","shop_2","shop_3","shop_4","shop_5","shop_6"] # shop names as string
second_shop = []
for first, second in shop_pairs: 
    st,p_v=stats.mannwhitneyu(df[(df["shop_id"] == first)]["price"].values.tolist(), 
                          df[(df["shop_id"] == second)]["price"].values.tolist())
    test_stat.append(st.round(4))
    p_values.append(p_v.round(4))
    first_shop.append(first) 
    second_shop.append(second)
p_value_table = pd.DataFrame({"first_shop":first_shop,"second_shop":second_shop,"test_stat": test_stat, "p_values":p_values})

##### Here we created this owesome dataframe below to find out which shop or shops are responsible from difference between prices accros shops

In [14]:
p_value_table

,first_shop,second_shop,test_stat,p_values
0,shop_1,shop_2,5010.0,0.0001
1,shop_1,shop_3,29424.0,0.4252
2,shop_1,shop_4,60158.0,0.0001
3,shop_1,shop_5,6121.0,0.3905
4,shop_1,shop_6,34006.0,0.4124
5,shop_2,shop_3,31313.0,0.0000
6,shop_2,shop_4,65110.0,0.0000
7,shop_2,shop_5,6575.5,0.0000
8,shop_2,shop_6,36586.0,0.0000
9,shop_3,shop_4,370034.5,0.0000


##### As you can see all possible combinatios of price comparison between shops included in the table. But we are interested in the ones that p_values less than 0,05
##### Lets filter them

In [15]:
p_value_table[p_value_table["p_values"]<0.05]

,first_shop,second_shop,test_stat,p_values
0,shop_1,shop_2,5010.0,0.0001
2,shop_1,shop_4,60158.0,0.0001
5,shop_2,shop_3,31313.0,0.0000
6,shop_2,shop_4,65110.0,0.0000
7,shop_2,shop_5,6575.5,0.0000
8,shop_2,shop_6,36586.0,0.0000
9,shop_3,shop_4,370034.5,0.0000
12,shop_4,shop_5,83360.5,0.0001
13,shop_4,shop_6,454601.0,0.0000


##### Here if you carefully look, you will notice that all the lines above contains shop_4 or shop_2. 

##### This means that shop_2 and shop_4 is responsible from price difference accros the shops. 

##### According to line 4 they are not alike each other too. 

##### For the rest of the shops we can not say that they are statisticly different from each other interms of price.



##### Now we know that the 2 shops mentioned above are different from other shops interms of price customers willing to pay.

##### As our goal is to ensure maximum profit we need to have different prices in these 2 shops as they are not alike with any of the shops including each other.

##### For other shops which are not statisticly different from each other interms of price, we can set the same price.

##### As company policy we can decide that item price should be same across the shops. But we are not going to do that.


### Confidence intervals

##### As we decided to go on with different prices lets determine different confidence intervals. We are going to use this confidence intervals to simulate our revenue

In [24]:
alike_shops = ["shop_1", "shop_3","shop_5", "shop_6"] # alike shops interms of price
shop_1356 = df[(df["shop_id"].isin(alike_shops))] # here we are gathering them to calculate confidence interval
shop_1356_con_int =sms.DescrStatsW(shop_1356["price"]).tconfint_mean() # here we are calculating confidence interval
shop_1356_con_int

(35.803488556367476, 36.7997387818449)

##### Here we calculated confidence interval for alike shops(shop_1, shop_3, shop_5, shop_6)

In [25]:
shop_2_con_int = sms.DescrStatsW(shop_2).tconfint_mean() 
shop_2_con_int

(33.244221031281384, 36.24683640338621)

##### Here we calculated confidence interval for shop_2

In [26]:
shop_4_con_int = sms.DescrStatsW(shop_4).tconfint_mean()
shop_4_con_int

(39.891734916786355, 41.15449478737718)

##### Here we calculated confidence interval for shop_4

### Simulations of Profit

#### We will calculate the revenue with the minimum and maximum values of the confidence intervals 

#### First lets calculate the revenue with max value of confidence intervals

##### Shop_1356 Revenue Calculation

In [27]:
freq_1356 = len(shop_1356[shop_1356["price"] >= shop_1356_con_int[1]]) # here we determined that how many people is willing to pay more than our max confidence interval 
revenue_1356 = freq_1356 * shop_1356_con_int[1] # here we multiplied what we found above with max confidence interval to calculate revenue
print(f'Revenue: {revenue_1356}')

Revenue: 10524.725291607641


##### Shop_2 Revenue Calculation

In [28]:
shop_2 = df[(df["shop_id"] == "shop_2")]
freq_2 = len(shop_2[shop_2["price"] >= shop_2_con_int[1]]) # here we determined that how many people is willing to pay more than our max confidence interval 
revenue_2 = freq_2 * shop_2_con_int[1] # here we multiplied what we found above with max confidence interval to calculate revenue
print(f'Revenue: {revenue_2}')

Revenue: 724.9367280677243


##### Shop_1356 Revenue Calculation

In [29]:
shop_4 = df[(df["shop_id"] == "shop_4")]
freq_4 = len(shop_4[shop_4["price"] >= shop_4_con_int[1]]) # here we determined that how many people is willing to pay more than our max confidence interval 
revenue_4 = freq_4 * shop_4_con_int[1] # here we multiplied what we found above with max confidence interval to calculate revenue
print(f'Revenue: {revenue_4}')

Revenue: 20412.629414539082


In [30]:
total_revenue_with_max = revenue_1356 + revenue_2 + revenue_4 # here we calculate total revenue
total_revenue_with_max.round(2)

31662.29

##### As you can see that our total revenue is 31662.29 for the scenario that we decided to go on max value of confidence interval 

##### Now lets calculate the profit with min value of confidence intervals

##### Shop_1356 Revenue Calculation

In [31]:
freq_1356 = len(shop_1356[shop_1356["price"] >= shop_1356_con_int[0]]) # here we determined that how many people is willing to pay more than our max confidence interval 
revenue_1356 = freq_1356 * shop_1356_con_int[0] # here we multiplied what we found above with max confidence interval to calculate revenue
print(f'Revenue: {revenue_1356}')

Revenue: 12602.827971841352


##### Shop_2 Revenue Calculation

In [32]:
shop_2 = df[(df["shop_id"] == "shop_2")]
freq_2 = len(shop_2[shop_2["price"] >= shop_2_con_int[0]]) # here we determined that how many people is willing to pay more than our max confidence interval 
revenue_2 = freq_2 * shop_2_con_int[0] # here we multiplied what we found above with max confidence interval to calculate revenue
print(f'Revenue: {revenue_2}')

Revenue: 1662.2110515640693


##### Shop_1356 Revenue Calculation

In [33]:
shop_4 = df[(df["shop_id"] == "shop_4")]
freq_4 = len(shop_4[shop_4["price"] >= shop_4_con_int[0]]) # here we determined that how many people is willing to pay more than our max confidence interval 
revenue_4 = freq_4 * shop_4_con_int[0] # here we multiplied what we found above with max confidence interval to calculate revenue
print(f'Revenue: {revenue_4}')

Revenue: 21780.88726456535


In [34]:
total_revenue_with_min = revenue_1356 + revenue_2 + revenue_4 # here we calculate total revenue
total_revenue_with_min.round(2)

36045.93

##### As you can see that our total revenue is 36045.93 for the scenario that we decided to go on min value of confidence interval 

### Conclusion

##### In this project our goal was to set optimum price for a particular item being sold in different shops

##### We evaluated the survey results by applying ANOVA test to see wheter there is a statisticly significant difference between prices people wish to pay across the shops

##### As we go deep in the source of this difference, found that 2 shops is are not alike with any other shops according to our Mann–Whitney U test result

##### Then we determined to confidence intervals to finally simulate the sales according to maximum and minimum values of confidence interval

##### According to our foundings above, setting the price to minimum value of confidence interval resulted in higher revenue amount. 
##### However it does not mean that it is more profitable to sell this product with min value of confidence interval as we do not know what is the cost for this product.
##### But our job is done here. We made the necessary tests and provided the information needed.